In [148]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy
from tqdm.notebook import tqdm


In [150]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32
EPOCHS = 12
LR = 0.001


In [152]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = datasets.MNIST(root="./dataprogram6/", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./dataprogram6/", train=False, transform=transform, download=True)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [154]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 6, 5), nn.Tanh(), nn.AvgPool2d(2, 2),
            nn.Conv2d(6, 16, 5), nn.Tanh(), nn.AvgPool2d(2, 2)
        )
        self.flattened_size = self._get_flattened_size()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_size, 120), nn.Tanh(),
            nn.Linear(120, 84), nn.Tanh(),
            nn.Linear(84, 10)
        )

    def _get_flattened_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, 28, 28)
            return self.feature_extractor(dummy_input).numel()

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x


In [156]:
model = LeNet5().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
accuracy = Accuracy(task="multiclass", num_classes=10).to(device)



In [162]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS)):
    ###### Training Loop ######
    model.train()
    train_loss, train_acc = 0.0, 0.0
    
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        y_pred = model(X)  # Forward pass
        loss = loss_fn(y_pred, y)  # Compute loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(y_pred.argmax(dim=1), y).item()  # Convert accuracy to scalar

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    ###### Validation Loop ######
    val_loss, val_acc = 0.0, 0.0
    model.eval()

    with torch.no_grad():  # Use torch.no_grad() to avoid inplace update errors
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)

            val_loss += loss.item()
            val_acc += accuracy(y_pred.argmax(dim=1), y).item()  # Convert accuracy to scalar

    val_loss /= len(val_loader)
    val_acc /= len(val_loader)

    

    ###### Print Progress ######
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.5f} | Train Acc: {train_acc:.5f} | Val Loss: {val_loss:.5f} | Val Acc: {val_acc:.5f}")




  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/12 | Train Loss: 0.05125 | Train Acc: 0.98384 | Val Loss: 0.05921 | Val Acc: 0.98138
Epoch 2/12 | Train Loss: 0.04013 | Train Acc: 0.98774 | Val Loss: 0.04813 | Val Acc: 0.98521
Epoch 3/12 | Train Loss: 0.03335 | Train Acc: 0.98937 | Val Loss: 0.04702 | Val Acc: 0.98537
Epoch 4/12 | Train Loss: 0.02754 | Train Acc: 0.99119 | Val Loss: 0.04780 | Val Acc: 0.98321
Epoch 5/12 | Train Loss: 0.02435 | Train Acc: 0.99200 | Val Loss: 0.05517 | Val Acc: 0.98288
Epoch 6/12 | Train Loss: 0.02005 | Train Acc: 0.99339 | Val Loss: 0.03944 | Val Acc: 0.98870
Epoch 7/12 | Train Loss: 0.01766 | Train Acc: 0.99424 | Val Loss: 0.04398 | Val Acc: 0.98820
Epoch 8/12 | Train Loss: 0.01714 | Train Acc: 0.99437 | Val Loss: 0.05091 | Val Acc: 0.98620
Epoch 9/12 | Train Loss: 0.01408 | Train Acc: 0.99522 | Val Loss: 0.05121 | Val Acc: 0.98920
Epoch 10/12 | Train Loss: 0.01279 | Train Acc: 0.99563 | Val Loss: 0.05171 | Val Acc: 0.98737
Epoch 11/12 | Train Loss: 0.01241 | Train Acc: 0.99587 | Val Loss: 0.